In [66]:
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks


import warnings
warnings.filterwarnings("ignore")

# 1. Read your Hopsworks API Key from a file or environment variable
#    For example, if stored in 'hopsworks-api-key.txt'
with open('../data/hopsworks-api-key.txt', 'r') as f:
    api_key = f.read().strip()

# 2. Set the environment variable for Hopsworks
os.environ["HOPSWORKS_API_KEY"] = api_key

project = hopsworks.login()
fs = project.get_feature_store()

# Retrieve feature groups for Clash Royale
player_stats_fg = fs.get_feature_group(
    name='clash_royale_onehotencoding',
    version=5,
)
game_events_fg = fs.get_feature_group(
    name='clash_royale_dataset_onehotencoding',
    version=5,
)

df = player_stats_fg.read()




'''
# Retrieve features from feature groups
player_stats = player_stats_fg.select_all()
game_events = game_events_fg.select_all()

# Explicitly prefix column names to avoid ambiguity
player_stats = player_stats.with_column_renamed("player_name", "player_stats_player_name")
game_events = game_events.with_column_renamed("player_name", "game_events_player_name")

# Join the feature groups on the prefixed columns
selected_features = player_stats.join(
    game_events,
    on=["player_stats_player_name", "game_events_player_name"]
)

# Display the first 10 rows of the joined features
selected_features.show(10)


feature_view = fs.get_or_create_feature_view(
    name='clash_royale_fv',
    description="Clash Royale game prediction features",
    version=1,
    labels=['player_name'],
    #query=selected_features,
)
'''

2025-01-06 13:36:22,869 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-06 13:36:22,880 INFO: Initializing external client
2025-01-06 13:36:22,880 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-06 13:36:24,315 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164438
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.56s) 


'\n# Retrieve features from feature groups\nplayer_stats = player_stats_fg.select_all()\ngame_events = game_events_fg.select_all()\n\n# Explicitly prefix column names to avoid ambiguity\nplayer_stats = player_stats.with_column_renamed("player_name", "player_stats_player_name")\ngame_events = game_events.with_column_renamed("player_name", "game_events_player_name")\n\n# Join the feature groups on the prefixed columns\nselected_features = player_stats.join(\n    game_events,\n    on=["player_stats_player_name", "game_events_player_name"]\n)\n\n# Display the first 10 rows of the joined features\nselected_features.show(10)\n\n\nfeature_view = fs.get_or_create_feature_view(\n    name=\'clash_royale_fv\',\n    description="Clash Royale game prediction features",\n    version=1,\n    labels=[\'player_name\'],\n    #query=selected_features,\n)\n'

In [67]:
dfstats = player_stats_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.48s) 


In [68]:
dfdata = game_events_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.18s) 


In [51]:
df_1 = dfdata.iloc[:500,:]
df_2 = dfdata.iloc[500:,:]

In [70]:
selected_features = game_events_fg.select(['deck','deck2','result'])

In [72]:
feature_view = fs.get_or_create_feature_view(
    name='clashroyale_fv',
    description="deck features",
    version=3,
    labels=['result'],
    query=selected_features,
)

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1164438/featurestores/1155141/featureview). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270082,"usrMsg":"Feature: deck not found in feature group: clash_royale_dataset_onehotencoding","errorMsg":"Feature does not exist"}', error code: 270082, error msg: Feature does not exist, user msg: Feature: deck not found in feature group: clash_royale_dataset_onehotencoding

In [54]:
#feature_view.read()

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = feature_view.train_test_split(test_size=0.2)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.58s) 


In [56]:
X_train

,deck,deck2
0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
615,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
618,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
622,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
623,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [64]:
print(X_train.isna().sum())


0      0
1      0
2      0
3      0
4      0
      ..
176    0
177    0
178    0
179    0
180    0
Length: 362, dtype: int64


In [57]:
y_train

,result,result2
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
615,1,0
618,1,0
622,1,0
623,1,0


In [58]:
# Example of flattening columns
deck_flat = pd.DataFrame(data=list(X_train['deck']), index=X_train.index)
deck2_flat = pd.DataFrame(data=list(X_train['deck2']), index=X_train.index)

# Merge flattened columns into the dataset
X_train = pd.concat([X_train.drop(['deck', 'deck2'], axis=1), deck_flat, deck2_flat], axis=1)
X_test = pd.concat([X_test.drop(['deck', 'deck2'], axis=1), 
                    pd.DataFrame(data=list(X_test['deck']), index=X_test.index), 
                    pd.DataFrame(data=list(X_test['deck2']), index=X_test.index)], axis=1)


In [62]:
print(X_test.dtypes)


0      int64
1      int64
2      int64
3      int64
4      int64
       ...  
176    int64
177    int64
178    int64
179    int64
180    int64
Length: 362, dtype: object


In [60]:
# Drop unnecessary columns
#train_features = X_train.drop(['date', 'player_id'], axis=1)
#test_features = X_test.drop(['date', 'player_id'], axis=1)

# Create and train the XGBoost Regressor
#xgb_regressor = XGBRegressor()
xgb_regressor = XGBRegressor(enable_categorical=True)
xgb_regressor.fit(X_train, y_train)

# Predict game outcomes on the test set
y_pred = xgb_regressor.predict(X_test)

# Calculate and print metrics
mse = mean_squared_error(y_test.iloc[:, 0], y_pred)
print("MSE:", mse)

r2 = r2_score(y_test.iloc[:, 0], y_pred)
print("R squared:", r2)

# Save predictions to DataFrame
df = y_test
df['predicted_game_outcome'] = y_pred

# Create directory for model artifacts
model_dir = "clash_royale_model"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
images_dir = model_dir + "/images"
if not os.path.exists(images_dir):
    os.mkdir(images_dir)

# Visualize and save feature importance
plot_importance(xgb_regressor, max_num_features=4)
feature_importance_path = images_dir + "/feature_importance.png"
plt.savefig(feature_importance_path)
plt.show()

from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Define input and output schemas
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)
schema_dict = model_schema.to_dict()

# Save the XGBoost model
xgb_regressor.save_model(model_dir + "/model.json")
mr = project.get_model_registry()

# Register the model in the Hopsworks model registry
cr_model = mr.python.create_model(
    name="clash_royale_xgboost_model",
    metrics={"mse": mse, "r2": r2},
    model_schema=model_schema,
    input_example=X_test.sample().values,
    description="Clash Royale game outcome predictor",
)

# Save the model artifacts to the registry
cr_model.save(model_dir)


XGBoostError: [12:57:01] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\data\array_interface.h:218: Check failed: m == 1 || n == 1: 